# SQL TWO
## Análise de Dados com SQL
### Aula 2 - OVER, PARTITION BY

### 1 - Diretório
#### Uma boa prática é inicialmente limpar o diretório, ou seja, excluir os arquivos (caso já tenham sido criados)

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/nossa_loja

res3: Boolean = false

### 2 - Tabela nossa_loja
#### Vamos criar uma tabela com dados fictícios para nossos tratamentos neste notebook. Utilizaremos DROP e CREATE TABLE para isso.

In [0]:
-- 1 - Criação da base de dados
DROP TABLE IF EXISTS NOSSA_LOJA;

CREATE TABLE NOSSA_LOJA(
  id_estado INT,
  nome_estado VARCHAR(30),
  regiao VARCHAR(30),
  qtd_vendida FLOAT
);

### 3 - Inserindo dados das lojas na tabela:
 - INSERT INTO
 - nome da tabela
    - (colunas da tabela)
 - VALUES
    - (dados de cada coluna separados por vírgula)

In [0]:
-- 2 - Inserindo dados na tabela
INSERT INTO
NOSSA_LOJA
VALUES
(1, "São Paulo", "Sudeste", 1800),
(2, "Rio de Janeiro", "Sudeste", 800),
(3, "Minas Gerais", "Sudeste", 2300),
(4, "Espírito Santo", "Sudeste", 1800),
(5, "Paraná", "Sul", 700),
(6, "Santa Catarina", "Sul", 2100),
(7, "Rio Grande do Sul", "Sul", 990),
(8, "Amazonas", "Norte", 4200),
(9, "Pará", "Norte", 3200),
(10, "Acre", "Norte", 1500),
(11, "Bahia", "Nordeste", 1910),
(12, "Pernambuco", "Nordeste", 2380);

num_affected_rows,num_inserted_rows
12,12


### 4 - Visualizando a base de dados

In [0]:
-- 3 - Visualização da tabela
SELECT * FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida
1,São Paulo,Sudeste,1800.0
2,Rio de Janeiro,Sudeste,800.0
3,Minas Gerais,Sudeste,2300.0
4,Espírito Santo,Sudeste,1800.0
5,Paraná,Sul,700.0
6,Santa Catarina,Sul,2100.0
7,Rio Grande do Sul,Sul,990.0
8,Amazonas,Norte,4200.0
9,Pará,Norte,3200.0
10,Acre,Norte,1500.0


### 5 - SUM
#### Podemos verificar o valor total das vendas das lojas na coluna qtd_vendida.

In [0]:
-- 4 - Verificando o valor total da coluna qtd_vendida
SELECT SUM(qtd_vendida) AS soma_total_de_vendas
FROM NOSSA_LOJA;

soma_total_de_vendas
23680.0


### 6 - GROUP BY ALL x OVER + DISTINCT
#### Calculando a soma da quantidade vendida por região com GROUP BY ALL.

In [0]:
-- 5 - Calculando o total vendido por região com Sum e Group By
SELECT 
  regiao,
  SUM(qtd_vendida) 
FROM NOSSA_LOJA
GROUP BY ALL
ORDER BY regiao;

regiao,sum(qtd_vendida)
Nordeste,4290.0
Norte,8900.0
Sudeste,6700.0
Sul,3790.0


### 7 - SUM, DISTINCT, OVER

In [0]:
-- 6 - Calculando o total vendido por região com DISTINCT, SUM e OVER
SELECT 
  DISTINCT regiao,
  SUM(qtd_vendida) OVER(PARTITION BY regiao) AS sum_qtd_vendida
FROM NOSSA_LOJA;

regiao,sum_qtd_vendida
Nordeste,4290.0
Norte,8900.0
Sudeste,6700.0
Sul,3790.0


### 8 - SUM, OVER sem distinct

In [0]:
-- 7 - Calculando o total vendido por região com SUM e OVER, sem DISTINCT
SELECT 
  regiao,
  SUM(qtd_vendida) OVER(PARTITION BY regiao) AS sum_qtd_vendida
FROM NOSSA_LOJA;

regiao,sum_qtd_vendida
Nordeste,4290.0
Nordeste,4290.0
Norte,8900.0
Norte,8900.0
Norte,8900.0
Sudeste,6700.0
Sudeste,6700.0
Sudeste,6700.0
Sudeste,6700.0
Sul,3790.0


### 9 - SUM, OVER sem partition by

In [0]:
-- 8 - Calculando o percentual do total vendido por cada estado pelo total da base
SELECT 
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida, 
  SUM(qtd_vendida) OVER() AS sum_qtd_vendida,
  ROUND(qtd_vendida / sum_qtd_vendida * 100, 2) AS pct_qtd_vendida
FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida,sum_qtd_vendida,pct_qtd_vendida
1,São Paulo,Sudeste,1800.0,23680.0,7.6
2,Rio de Janeiro,Sudeste,800.0,23680.0,3.38
3,Minas Gerais,Sudeste,2300.0,23680.0,9.71
4,Espírito Santo,Sudeste,1800.0,23680.0,7.6
5,Paraná,Sul,700.0,23680.0,2.96
6,Santa Catarina,Sul,2100.0,23680.0,8.87
7,Rio Grande do Sul,Sul,990.0,23680.0,4.18
8,Amazonas,Norte,4200.0,23680.0,17.74
9,Pará,Norte,3200.0,23680.0,13.51
10,Acre,Norte,1500.0,23680.0,6.33


### 10 - SUM, OVER + PARTITION BY

In [0]:
-- 9 - Calculando o total vendido por região com SUM e OVER para todas as colunas por região
SELECT 
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida, 
  SUM(qtd_vendida) OVER(PARTITION BY regiao) AS sum_qtd_vendida,
  ROUND(qtd_vendida / sum_qtd_vendida * 100, 2) AS pct_qtd_vendida
FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida,sum_qtd_vendida,pct_qtd_vendida
11,Bahia,Nordeste,1910.0,4290.0,44.52
12,Pernambuco,Nordeste,2380.0,4290.0,55.48
8,Amazonas,Norte,4200.0,8900.0,47.19
9,Pará,Norte,3200.0,8900.0,35.96
10,Acre,Norte,1500.0,8900.0,16.85
1,São Paulo,Sudeste,1800.0,6700.0,26.87
2,Rio de Janeiro,Sudeste,800.0,6700.0,11.94
3,Minas Gerais,Sudeste,2300.0,6700.0,34.33
4,Espírito Santo,Sudeste,1800.0,6700.0,26.87
5,Paraná,Sul,700.0,3790.0,18.47


### 11 - PARTITION BY x ORDER BY

In [0]:
-- 10 - Calculando o percentual do total vendido por cada estado por região
SELECT 
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida, 
  SUM(qtd_vendida) OVER(PARTITION BY regiao) AS sum_qtd_vendida,
  ROUND(qtd_vendida / sum_qtd_vendida * 100, 2) AS pct_qtd_vendida
FROM NOSSA_LOJA
ORDER BY id_estado;

id_estado,nome_estado,regiao,qtd_vendida,sum_qtd_vendida,pct_qtd_vendida
1,São Paulo,Sudeste,1800.0,6700.0,26.87
2,Rio de Janeiro,Sudeste,800.0,6700.0,11.94
3,Minas Gerais,Sudeste,2300.0,6700.0,34.33
4,Espírito Santo,Sudeste,1800.0,6700.0,26.87
5,Paraná,Sul,700.0,3790.0,18.47
6,Santa Catarina,Sul,2100.0,3790.0,55.41
7,Rio Grande do Sul,Sul,990.0,3790.0,26.12
8,Amazonas,Norte,4200.0,8900.0,47.19
9,Pará,Norte,3200.0,8900.0,35.96
10,Acre,Norte,1500.0,8900.0,16.85


In [0]:
-- 11 - Calculando o percentual do total vendido por cada estado por região???!!!
SELECT 
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida, 
  SUM(qtd_vendida) OVER(PARTITION BY regiao ORDER BY id_estado) AS sum_qtd_vendida,
  ROUND(qtd_vendida / sum_qtd_vendida * 100, 2) AS pct_qtd_vendida
FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida,sum_qtd_vendida,pct_qtd_vendida
11,Bahia,Nordeste,1910.0,1910.0,100.0
12,Pernambuco,Nordeste,2380.0,4290.0,55.48
8,Amazonas,Norte,4200.0,4200.0,100.0
9,Pará,Norte,3200.0,7400.0,43.24
10,Acre,Norte,1500.0,8900.0,16.85
1,São Paulo,Sudeste,1800.0,1800.0,100.0
2,Rio de Janeiro,Sudeste,800.0,2600.0,30.77
3,Minas Gerais,Sudeste,2300.0,4900.0,46.94
4,Espírito Santo,Sudeste,1800.0,6700.0,26.87
5,Paraná,Sul,700.0,700.0,100.0


### 12 - CTE, SUM, OVER, PARTITION BY

In [0]:
-- 12 - Calculando o percentual do total vendido por cada estado por região

WITH CTE AS
(
SELECT 
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida, 
  SUM(qtd_vendida) OVER(PARTITION BY regiao) AS sum_qtd_vendida,
  ROUND(qtd_vendida / sum_qtd_vendida * 100, 2) AS pct_qtd_vendida
FROM NOSSA_LOJA
ORDER BY regiao ASC,
         pct_qtd_vendida DESC
)

SELECT
  nome_estado AS estado,
  regiao,
  pct_qtd_vendida AS pct_estadual_regional_vendas
FROM CTE;

estado,regiao,pct_estadual_regional_vendas
Pernambuco,Nordeste,55.48
Bahia,Nordeste,44.52
Amazonas,Norte,47.19
Pará,Norte,35.96
Acre,Norte,16.85
Minas Gerais,Sudeste,34.33
São Paulo,Sudeste,26.87
Espírito Santo,Sudeste,26.87
Rio de Janeiro,Sudeste,11.94
Santa Catarina,Sul,55.41
